# Gradium API

This API exposes our Text-to-Speech and Speech-to-Text models. The TTS model offers low-latency, high-quality & natural sounding outputs in 5 languages.

A pre-requisite is to install the gradium wheel, the cell below will handle this.

In [1]:
!pip install gradium

One will need to either set `GRADIUM_API_KEY` or pass the api key directly to `GradiumClient`

In [3]:
import os
# Use your gradium API key directly here.
os.environ["GRADIUM_API_KEY"] = 'gsk...'

# Or use colab to manage your secrets by uncommenting the two lines below.
# from google.colab import userdata
# os.environ["GRADIUM_API_KEY"] = userdata.get('GRADIUM_API_KEY')

In [4]:
from gradium import GradiumClient
client = GradiumClient(base_url='eu.api.gradium.ai/api')

## Text-to-speech

3 modes are possible:
1. Streaming input and output (`gradium.speech.tts_stream`)
2. Non-streaming input and streaming output (`gradium.speech.tts_stream`)
3. Non-streaming input and output (`gradium.speech.tts`)

The simplest is to use the non-streaming mode. You can find a list of the available voices in our [documentation website](https://gradium.ai/api_docs.html).

In [5]:
from IPython import display

audio = await client.tts(
    setup={'voice_id': 'YTpq7expH9539ERJ', 'output_format': 'wav'},
    text='Hello, this is a test of the Gradium text-to-speech system.'
)

display.display(display.Audio(audio.raw_data))

In [6]:
audio = await client.tts(
    setup={'voice_id': 'axlOaUiFyOZhy4nv', 'output_format': 'wav'},
    text='Bonjour, ceci est un test du système de synthèse vocale de Gradium. Merci de bien vouloir respecter la consigne.'
)

display.display(display.Audio(audio.raw_data))

# Check your credits

The command below reports the amount of available credits.
One credit is equivalent to one character of TTS, so one minute of generated audio is roughly equivalent to 750 credits.

In [7]:
await client.credits()

{'remaining_credits': 27504135,
 'allocated_credits': 28000000,
 'billing_period': '11-2025',
 'next_rollover_date': None,
 'plan_name': ''}

## Create a voice

Creating a voice returns the voice_id (and an error if any). The voice cloning is based on a sample of ten seconds and call below lets you pick the beginning of the sample.

- Ensure that there is no background noise.
- Cleanup the audio with a tool such as https://podcast.adobe.com/en/enhance
- Ideally use some professional microphone.

A known issue is that when generating audio for a text that starts with the same word as cloning sample, it's likely for the cloning sample to be returned by the TTS. Because of this, it's better not to start the cloning sample with common words such as "Hello" etc.


In [8]:
!wget https://github.com/kyutai-labs/moshi/raw/refs/heads/main/data/sample_fr_hibiki_crepes.mp3 -O sample.mp3

import gradium
voice = await client.voice_create(
    'sample.mp3',
    name='someone',
    description='Someone with a nice voice',
    start_s=0,  # The cloned snippet goes from start_s to start_s + 10 seconds.
)
voice

--2025-12-09 10:50:32--  https://github.com/kyutai-labs/moshi/raw/refs/heads/main/data/sample_fr_hibiki_crepes.mp3
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kyutai-labs/moshi/refs/heads/main/data/sample_fr_hibiki_crepes.mp3 [following]
--2025-12-09 10:50:32--  https://raw.githubusercontent.com/kyutai-labs/moshi/refs/heads/main/data/sample_fr_hibiki_crepes.mp3
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 759450 (742K) [audio/mpeg]
Saving to: ‘sample.mp3’

sample.mp3          100%[===================>] 741.65K  --.-KB/s    in 0.04s   

2025-12-09 10:50:32 (16.7 MB/s) - ‘sample.mp3’ saved 

{'uid': 'Zwg3Bg18lmN3ErO5',
 'error': 'Please wait a bit till the voice is ready to be used.',
 'was_updated': True}

Double check the available custom voices:

In [9]:
await client.voice_list()

[{'uid': 'jS9r5KTW6RFI3svg',
  'name': '4193_3103_000004-0001.wav',
  'description': None,
  'language': None,
  'start_s': 0.0,
  'stop_s': 10.0,
  'filename': '4193_3103_000004-0001.wav'},
 {'uid': 'mRhtk-bHd1n6GN96',
  'name': 'p341_024.wav',
  'description': None,
  'language': None,
  'start_s': 0.0,
  'stop_s': 10.0,
  'filename': 'p341_024.wav'},
 {'uid': 'onC6nVazpWhHI1MW',
  'name': 'p247_019.wav',
  'description': None,
  'language': None,
  'start_s': 0.0,
  'stop_s': 10.0,
  'filename': 'p247_019.wav'},
 {'uid': 'lXQVK-UvMLAeVv3b',
  'name': 'p333_023.wav',
  'description': None,
  'language': None,
  'start_s': 0.0,
  'stop_s': 10.0,
  'filename': 'p333_023.wav'},
 {'uid': 's4c9nmxarWyph00e',
  'name': 'p304_019.wav',
  'description': None,
  'language': None,
  'start_s': 0.0,
  'stop_s': 10.0,
  'filename': 'p304_019.wav'},
 {'uid': 'p3LVdyZT7F4dxO9w',
  'name': 'p335_011.wav',
  'description': None,
  'language': None,
  'start_s': 0.0,
  'stop_s': 10.0,
  'filename': '

Generate some audio using this voice.

In [10]:
audio = await client.tts(
    setup={
        'voice_id': voice['uid'],
        'output_format': 'wav',
    },
    text='Ceci est un message venu du fin fond d\' un GPU'
)

display.display(display.Audio(audio.raw_data))

## Speech-to-Text

The Speech-to-Text model converts audio input to text. It also returns some Voice Activity Detection (VAD) information which is helpful to determine end of turns.

This model is fully streaming, one can submit a chunck of audio at a time. When using the `pcm` input type, the data is expected to be sampled at 24kHz and use int-16 little endian encoded values.

In [11]:
# We install sphn so that it's easy to load the audio data
!pip install sphn
!wget https://github.com/kyutai-labs/delayed-streams-modeling/raw/refs/heads/main/audio/bria.mp3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 75.2 MB/s eta 0:00:00
--2025-12-09 10:51:13--  https://github.com/kyutai-labs/delayed-streams-modeling/raw/refs/heads/main/audio/bria.mp3
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kyutai-labs/delayed-streams-modeling/refs/heads/main/audio/bria.mp3 [following]
--2025-12-09 10:51:13--  https://raw.githubusercontent.com/kyutai-labs/delayed-streams-modeling/refs/heads/main/audio/bria.mp3
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 717635 (701K) [audio/mpeg]
Saving to: ‘bria.mp3’

bria.mp3            100%[===================>] 700.82K

In [12]:
import sphn
import numpy as np
audio_data, _ = sphn.read("bria.mp3", sample_rate=24000)
audio_data = (audio_data[0] * 32768).astype(np.int16)

In [13]:
async def audio_generator(audio_data, chunk_size=1920):
    for i in range(0, len(audio_data), chunk_size):
        yield audio_data[i : i + chunk_size]

# Create STT stream
stream = await client.stt_stream(
    {"model_name": "default", "input_format": "pcm"},
    audio_generator(audio_data),
)

# Process transcription results
async for message in stream.iter_text():
    print(message)

TextWithTimestamps(text='In the', start_s=0.08, stop_s=0.08)
TextWithTimestamps(text='heart of', start_s=0.4, stop_s=0.4)
TextWithTimestamps(text='an', start_s=0.64, stop_s=0.64)
TextWithTimestamps(text='ancient', start_s=0.88, stop_s=0.88)
TextWithTimestamps(text='forest,', start_s=1.28, stop_s=1.28)
TextWithTimestamps(text='where the', start_s=1.76, stop_s=1.76)
TextWithTimestamps(text='trees', start_s=2.1599998, stop_s=2.1599998)
TextWithTimestamps(text='whispered secrets of the', start_s=2.72, stop_s=2.72)
TextWithTimestamps(text='past,', start_s=3.6, stop_s=3.6)
TextWithTimestamps(text='there', start_s=4.24, stop_s=4.24)
TextWithTimestamps(text='lived', start_s=4.3999996, stop_s=4.3999996)
TextWithTimestamps(text='a', start_s=4.64, stop_s=4.64)
TextWithTimestamps(text='peculiar', start_s=4.96, stop_s=4.96)
TextWithTimestamps(text='rabbit', start_s=5.44, stop_s=5.44)
TextWithTimestamps(text='named', start_s=5.84, stop_s=5.84)
TextWithTimestamps(text='Luna.', start_s=6.1600003, stop